# Scraping Fox News

First, I scraped articles from Fox News' climate change section. In the cell below, I create a function that allows me to load articles on the climate change section. This can be done by manipulating the offset parameter. I also include the JSON decoder in order to simplify the formatting of the page.

In [3915]:
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize

def fox_df(offset):
    url = ("https://www.foxnews.com/api/article-search?isCategory=false&isTag=true&isKeyword=false&"
    "isFixed=false&isFeedUrl=false&searchSelected=fox-news%2Fus%2Fenvironment%2Fclimate-change&contentTypes=%7B"
    "%22interactive%22:true,%22slideshow%22:true,%22video%22:true,%22article%22:true%7D&"
    "size=30&offset=0")
    
    url = url.replace('offset=0', 'offset=%s' % offset)
    
    r_fox = requests.get(url)
    new_df_fox = json_normalize(r_fox.json())
    
    return new_df_fox

Next, I generate a loop that loads as many articles as possible. To ensure this, I load 30 different articles 1000 times. The articles are in chronological order, so, the first 30 loaded, for example, will be the 30 most recently posted articles on Fox News. These articles are then all store in a dataframe, foxdf.

In [3916]:
from time import sleep # for pausing

# create empty dataframe to store results
foxdf = pd.DataFrame() 

# Create a loop that counts up by 30.
for offset in range(0, 1000, 30):
    new_df = fox_df(offset)
    
    # Add the new results to the existing database
    foxdf = foxdf.append(new_df, ignore_index=True)
    
    # Pause for three seconds to be polite to the web server
    sleep(3)

The length of the dataframe indicates that 284 articles were loaded, meaning that only 284 were avaiable on Fox News. This should be ample for this analysis.

In [3917]:
len(foxdf)

284

The dataframe below shows that numerous variables were stored. However, in my opinion, none of these varibles will serve as very good predictors for my model. For example, descriptions and titles do not seem to contain enough information to differentiate between Fox News and Huffington Post. As a result, I will retrieve the text of the articles in the subsequent steps since these will be rich with information.

In [3918]:
foxdf

,category.name,category.url,description,duration,imageUrl,isBreaking,isLive,lastPublishedDate,publicationDate,title,url
0,Media,/category/entertainment/media,An NBC News reporter raised eyebrows and promp...,,https://a57.foxnews.com/static.foxnews.com/fox...,False,False,2019-05-07T07:37:21-04:00,2019-05-07T07:37:21-04:00,NBC reporter scolded on air after comparing UN...,/entertainment/msnbc-climate-change-alien-inva...
1,OPINION,/category/opinion,Democratic presidential candidate Beto O'Rourk...,,https://a57.foxnews.com/static.foxnews.com/fox...,False,False,2019-04-30T22:17:40-04:00,2019-04-30T22:17:40-04:00,Gutfeld on Beto's $5 trillion plan to save the...,/opinion/gutfeld-on-betos-5-trillion-plan-to-s...
2,Fox News Flash,/category/topic/fox-news-flash,"""The Five"" host Jesse Watters had some tough w...",,https://a57.foxnews.com/static.foxnews.com/fox...,False,False,2019-04-30T18:34:44-04:00,2019-04-30T18:34:44-04:00,Watters on Beto's climate plan: He's a 'scam a...,/politics/watters-on-betos-climate-plan-hes-a-...
3,OPINION,/category/opinion,Democrats who truly wish to address climate an...,,https://a57.foxnews.com/static.foxnews.com/fox...,False,False,2019-04-30T12:08:56-04:00,2019-04-30T12:08:56-04:00,Mandy Gunasekara: Democrats should back Trump'...,/opinion/trump-paris-climate-accord-democrats
4,California,/category/us/us-regions/west/california,In the next few weeks California Gov. Gavin Ne...,,https://a57.foxnews.com/media2.foxnews.com/Bri...,False,False,2019-04-29T15:56:21-04:00,2019-04-29T15:56:21-04:00,California Gov. Newsom facing new pressure fro...,/politics/california-gov-newsom-facing-new-pre...
5,OPINION,/category/opinion,Climate change activists pulled an Earth Day s...,,https://a57.foxnews.com/media2.foxnews.com/Bri...,False,False,2019-04-24T16:02:08-04:00,2019-04-24T14:24:12-04:00,Tomi Lahren: California is the Golden State of...,/opinion/tomi-lahren-california-homeless-clima...
6,Alec Baldwin,/category/person/alec-baldwin,Actor Alec Baldwin said Tuesday that his passi...,,https://a57.foxnews.com/media2.foxnews.com/Bri...,False,False,2019-04-24T04:13:07-04:00,2019-04-24T04:12:40-04:00,Alec Baldwin calls for action on climate chang...,/entertainment/alec-baldwin-is-passionate-abou...
7,Media,/category/entertainment/media,"Little has changed over at CBS, with the netwo...",,https://a57.foxnews.com/static.foxnews.com/fox...,False,False,2019-04-23T11:37:55-04:00,2019-04-23T11:37:55-04:00,CBS resurfaces 1970 'Act or Die' alarmist clim...,/entertainment/cbs-climate-change-earth-day-ga...
8,OPINION,/category/opinion,The hurricane pre-season hysteria is ramping u...,,https://a57.foxnews.com/static.foxnews.com/fox...,False,False,2019-04-23T06:42:31-04:00,2019-04-23T06:42:31-04:00,"Joe Bastardi: Hurricanes happen, whether they ...",/opinion/joe-bastardi-hurricanes-happen-whethe...
9,Climate Change,/category/us/environment/climate-change,Climate-change activists took over the Univers...,,https://a57.foxnews.com/media2.foxnews.com/Bri...,False,False,2019-04-22T19:10:27-04:00,2019-04-22T19:10:27-04:00,Protesters climb Universal Studios globe in Ca...,/us/activists-universal-studios-globe-californ...


In the dataframe above, publisher information is not clearly available. This is ultimately what I want to serve as my dependent variable in my model. Therefore, I will create a new column for publisher labeled, fox. 

In [3919]:
foxdf #add a column for publisher = 'fox'
foxdf['publisher']='fox'
foxdf

,category.name,category.url,description,duration,imageUrl,isBreaking,isLive,lastPublishedDate,publicationDate,title,url,publisher
0,Media,/category/entertainment/media,An NBC News reporter raised eyebrows and promp...,,https://a57.foxnews.com/static.foxnews.com/fox...,False,False,2019-05-07T07:37:21-04:00,2019-05-07T07:37:21-04:00,NBC reporter scolded on air after comparing UN...,/entertainment/msnbc-climate-change-alien-inva...,fox
1,OPINION,/category/opinion,Democratic presidential candidate Beto O'Rourk...,,https://a57.foxnews.com/static.foxnews.com/fox...,False,False,2019-04-30T22:17:40-04:00,2019-04-30T22:17:40-04:00,Gutfeld on Beto's $5 trillion plan to save the...,/opinion/gutfeld-on-betos-5-trillion-plan-to-s...,fox
2,Fox News Flash,/category/topic/fox-news-flash,"""The Five"" host Jesse Watters had some tough w...",,https://a57.foxnews.com/static.foxnews.com/fox...,False,False,2019-04-30T18:34:44-04:00,2019-04-30T18:34:44-04:00,Watters on Beto's climate plan: He's a 'scam a...,/politics/watters-on-betos-climate-plan-hes-a-...,fox
3,OPINION,/category/opinion,Democrats who truly wish to address climate an...,,https://a57.foxnews.com/static.foxnews.com/fox...,False,False,2019-04-30T12:08:56-04:00,2019-04-30T12:08:56-04:00,Mandy Gunasekara: Democrats should back Trump'...,/opinion/trump-paris-climate-accord-democrats,fox
4,California,/category/us/us-regions/west/california,In the next few weeks California Gov. Gavin Ne...,,https://a57.foxnews.com/media2.foxnews.com/Bri...,False,False,2019-04-29T15:56:21-04:00,2019-04-29T15:56:21-04:00,California Gov. Newsom facing new pressure fro...,/politics/california-gov-newsom-facing-new-pre...,fox
5,OPINION,/category/opinion,Climate change activists pulled an Earth Day s...,,https://a57.foxnews.com/media2.foxnews.com/Bri...,False,False,2019-04-24T16:02:08-04:00,2019-04-24T14:24:12-04:00,Tomi Lahren: California is the Golden State of...,/opinion/tomi-lahren-california-homeless-clima...,fox
6,Alec Baldwin,/category/person/alec-baldwin,Actor Alec Baldwin said Tuesday that his passi...,,https://a57.foxnews.com/media2.foxnews.com/Bri...,False,False,2019-04-24T04:13:07-04:00,2019-04-24T04:12:40-04:00,Alec Baldwin calls for action on climate chang...,/entertainment/alec-baldwin-is-passionate-abou...,fox
7,Media,/category/entertainment/media,"Little has changed over at CBS, with the netwo...",,https://a57.foxnews.com/static.foxnews.com/fox...,False,False,2019-04-23T11:37:55-04:00,2019-04-23T11:37:55-04:00,CBS resurfaces 1970 'Act or Die' alarmist clim...,/entertainment/cbs-climate-change-earth-day-ga...,fox
8,OPINION,/category/opinion,The hurricane pre-season hysteria is ramping u...,,https://a57.foxnews.com/static.foxnews.com/fox...,False,False,2019-04-23T06:42:31-04:00,2019-04-23T06:42:31-04:00,"Joe Bastardi: Hurricanes happen, whether they ...",/opinion/joe-bastardi-hurricanes-happen-whethe...,fox
9,Climate Change,/category/us/environment/climate-change,Climate-change activists took over the Univers...,,https://a57.foxnews.com/media2.foxnews.com/Bri...,False,False,2019-04-22T19:10:27-04:00,2019-04-22T19:10:27-04:00,Protesters climb Universal Studios globe in Ca...,/us/activists-universal-studios-globe-californ...,fox


To retrieve text from each individual article, I create a list containing the links from the 'url' column in the fox dataframe. I notice, however, that some of these links direct you to videos, which, of course, don't contain much text. Hence, I will first remove links from the dataframe that are videos. These are identified by containing "https" in the url. After doing so, the dataframe has been reduce to 222 articles. I also need to change the index of the newly created dataframe, rm_video, to have consective numbers or else the loop won't work.

In [3920]:
#remove rows that include video links in url
rm_video=foxdf[~foxdf.url.str.contains("https")]
rm_video
#change index to have consecutive numbers 
rm_video.index = range(len(rm_video))

Below, I create a loop that stores links in a list.

In [3921]:
url='https://www.foxnews.com' #create fox base url (domain name), we can append paths to this
links_fox=[]

#create a df that stores all of the links into one df
for i in range(0,len(rm_video)): 
        row=rm_video['url'][i] #path of each article
        full_url=url+row #append path to domain
        print(full_url)
        links_fox.append(full_url)
        print(i)

https://www.foxnews.com/entertainment/msnbc-climate-change-alien-invasion
0
https://www.foxnews.com/opinion/gutfeld-on-betos-5-trillion-plan-to-save-the-earth
1
https://www.foxnews.com/politics/watters-on-betos-climate-plan-hes-a-scam-artist-who-feels-guilty-about-americas-economic-success
2
https://www.foxnews.com/opinion/trump-paris-climate-accord-democrats
3
https://www.foxnews.com/politics/california-gov-newsom-facing-new-pressure-from-environmental-groups-to-block-drilling-fracking
4
https://www.foxnews.com/opinion/tomi-lahren-california-homeless-climate-change-activists
5
https://www.foxnews.com/entertainment/alec-baldwin-is-passionate-about-preserving-the-planet
6
https://www.foxnews.com/entertainment/cbs-climate-change-earth-day-gayle-king
7
https://www.foxnews.com/opinion/joe-bastardi-hurricanes-happen-whether-they-are-politically-exploited-by-climate-change-activists-or-not
8
https://www.foxnews.com/us/activists-universal-studios-globe-california-climate-change
9
https://www.

We can see that all links have been stored in a list.

In [3922]:
links_fox 

['https://www.foxnews.com/entertainment/msnbc-climate-change-alien-invasion',
 'https://www.foxnews.com/opinion/gutfeld-on-betos-5-trillion-plan-to-save-the-earth',
 'https://www.foxnews.com/politics/watters-on-betos-climate-plan-hes-a-scam-artist-who-feels-guilty-about-americas-economic-success',
 'https://www.foxnews.com/opinion/trump-paris-climate-accord-democrats',
 'https://www.foxnews.com/politics/california-gov-newsom-facing-new-pressure-from-environmental-groups-to-block-drilling-fracking',
 'https://www.foxnews.com/opinion/tomi-lahren-california-homeless-climate-change-activists',
 'https://www.foxnews.com/entertainment/alec-baldwin-is-passionate-about-preserving-the-planet',
 'https://www.foxnews.com/entertainment/cbs-climate-change-earth-day-gayle-king',
 'https://www.foxnews.com/opinion/joe-bastardi-hurricanes-happen-whether-they-are-politically-exploited-by-climate-change-activists-or-not',
 'https://www.foxnews.com/us/activists-universal-studios-globe-california-climate-c

Now that I have all article links, I can extract text from them. I do this using the newspaper.article package, which makes it easy to pull the aformentioned data from the links. In addition to this, I will extract article titles and descrptions from the links. This should provide some background for each article, making it easier to identify. This data will be stored in a new dataframe, fox. 

In [3908]:
from newspaper.article import ArticleException, ArticleDownloadState

article_info_fox=[] #empty list to store title/descpription of each individual article
df_fox=[] #combine data from all of these articles into one list then convert to df

for i in range(0,len(links_fox)): 
    article_fox = Article(links_fox[i]) #iterate through each article link
        
    slept = 0
    article_fox.download()
    while article_fox.download_state == ArticleDownloadState.NOT_STARTED:
    # Raise exception if article download state does not change after 10 seconds
        if slept > 9:
            raise ArticleException('Download never started')
    sleep(1)
    slept += 1
    
    article_fox.parse() #makes it easy to identify main components of article
       
    #to retrieve title/description/full text from each article + append publisher name
    article_info_fox={'title' : article_fox.title, 
                      'description': article_fox.meta_description,
                      'text':article_fox.text,
                      'publisher' : 'fox'} 

    df_fox.append(article_info_fox)
    fox = pd.DataFrame(df_fox)

Below, we see my final dataframe contains information for the entire text of each article as well as its description, publisher, and title. With this, I have completed the data retrieval process for Fox News.

In [3923]:
fox 

,description,publisher,text,title
0,Democratic presidential candidate Beto O'Rourk...,fox,Remember Beto?\n\nMonday former congressman fr...,Gutfeld on Beto's $5 trillion plan to save the...
1,"""The Five"" host Jesse Watters had some tough w...",fox,"""The Five"" co-host Jesse Watters had some toug...",Watters on Beto's climate plan: He's a 'scam a...
2,Democrats who truly wish to address climate an...,fox,"This week, Democrats in the House of Represent...",Mandy Gunasekara: Democrats should back Trump'...
3,In the next few weeks California Gov. Gavin Ne...,fox,"In the next few weeks, California Gov. Gavin N...",California Gov. Newsom facing new pressure fro...
4,Climate change activists pulled an Earth Day s...,fox,Here’s a headline for you: California climate ...,Tomi Lahren: California is the Golden State of...
5,Actor Alec Baldwin said Tuesday that his passi...,fox,Actor Alec Baldwin said Tuesday that his passi...,Alec Baldwin calls for action on climate chang...
6,"Little has changed over at CBS, with the netwo...",fox,CBS dug up for Earth Day a decades-old news cl...,CBS resurfaces 1970 'Act or Die' alarmist clim...
7,The hurricane pre-season hysteria is ramping u...,fox,The hurricane pre-season hysteria is ramping u...,"Joe Bastardi: Hurricanes happen, whether they ..."
8,Climate-change activists took over the Univers...,fox,Climate-change activists took over the Univers...,Protesters climb Universal Studios globe in Ca...
9,The protesters who glued themselves to trains ...,fox,The protesters who glued themselves to trains ...,Climate change protesters announce shift to po...


# Scraping Huffington Post

Next, I will scrape articles from the Huffington Post. As mentioned previously, the Huffington Post represents a left leaning media site, and, thus, will serve as the counterpart to Fox News. The goal is to end up with the same type of dataframe as Fox News with columns for description, publisher, text, and title. In the cell below, I use the Article command from the newspaper package. Moreover, I use the BeautifulSoup package to parse through each html page. I first define the function for collecting the information and then run it in a loop over all URLs. Since there are 26 articles per page, I need at least 9 pages to retrieve 222 articles--the same as Fox News. You can see that I loop through 11 pages (286 articles), to make sure I have enough articles just in case I have to remove those that are videos.

In [986]:
#Scraping Huffington Post

import requests
import json
import re
from newspaper import Article
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup

store_pages=[] #store html code for each page

for page_number in range(1,12): 
     
    page_number=str(page_number) #convert integer to string (Request command only reads strings)
    base_url_huff='https://www.huffpost.com/impact/topic/climate-change?page='
    r_huff = Request(base_url_huff + page_number, headers={'User-Agent': 'Mozilla/5.0'}) #To get around block
    webpage = urlopen(r_huff).read()
    page_soup=soup(webpage,"html.parser")
    store_pages.append(page_soup) #store each html parsed page in dataframe

    print('Now on page' + page_number)
    sleep(3)

Now on page1
Now on page2
Now on page3
Now on page4
Now on page5
Now on page6
Now on page7
Now on page8
Now on page9
Now on page10
Now on page11


Similar to what was done in the Fox News scraping, I retrieve article titles and descriptions by first retrieving links to each article. The stored links (285) can be seen below:

In [1071]:
links_huff = [] #empty list to store links
i=0
for i in range(0, len(store_pages)):
    for link in store_pages[i].findAll("a", {"class": "card__link yr-card-headline"}):
        links_huff.append(link.get('href'))
         
print(links_huff)

['https://www.huffpost.com/entry/jay-inslee-climate-policy-2020-candidates_n_5cca4231e4b0e4d7572d4973', 'https://www.huffpost.com/entry/beto-orourke-no-fossil-fuel-money-pledge_n_5cca3436e4b0913d078c3ab2', 'https://www.huffpost.com/entry/beto-orourke-climate-change_n_5cc8a757e4b0d123954bc820', 'https://www.huffpost.com/entry/the-gulf-belle-boggs-novel_n_5cc893b6e4b0e4d7572afbe4', 'https://www.huffpost.com/entry/beto-defends-climate-plan_n_5cc7c3bde4b053791149dccd', 'https://www.huffpost.com/entry/uk-labour-climate_n_5cc744f8e4b0537911494f9d', 'https://www.huffpost.com/entry/audrey-denney-2020_n_5cc49908e4b04eb7ff969323', 'https://www.huffpost.com/entry/bernie-sanders-green-new-deal_n_5cbe7a3ce4b0f7a84a741611', 'https://www.huffpost.com/entry/elizabeth-warren-climate-change-epa_n_5cbe635ae4b0f7a84a7404fb', 'https://www.huffpost.com/entry/jay-inslee-climate-debate_n_5cbe135ae4b00b3e70cda262', 'https://www.huffpost.com/entry/states-regulate-hfc-air-conditioners_b_5cbdd0cbe4b09e46a9f4e0eb'

Next, I retrieve article titles and descriptions from each link. Again, the article package makes it easy to identify main components of the article. Here, I'm interested in obtaining the article title, description, text. I also include a publisher column labeled, huff, which will serve as my dependent variable in my analysis in the next section. Lastly, I combine data from all of these articles into one list then convert to a dataframe.

In [1077]:
from newspaper.article import ArticleException, ArticleDownloadState

article_info_huff=[] #empty list to store title/descpription/text of each individual article
df_huff=[]

for i in range(0,len(links_huff)): 
    article_huff = Article(links_huff[i]) #iterate through each article link
        
    slept = 0
    article_huff.download()
    while article_huff.download_state == ArticleDownloadState.NOT_STARTED:
    # Raise exception if article download state does not change after 10 seconds
        if slept > 9:
            raise ArticleException('Download never started')
    sleep(1)
    slept += 1
    
    article_huff.parse() 
        
    article_info_huff={'title' : article_huff.title, 
                      'description': article_huff.meta_description,
                      'text': article_huff.text,
                      'publisher' : 'huff'} 

    df_huff.append(article_info_huff)
    huff = pd.DataFrame(df_huff)

In [1101]:
huff 

,description,publisher,text,title
0,The former congressman had defended taking con...,huff,Following criticism of his climate change plan...,Beto O’Rourke Will Sign 'No Fossil Fuel Money'...
1,"""Any city along the coast or on the ocean will...",huff,The plan drew some immediate criticism from ot...,Beto O'Rourke Urges Climate Change Action Duri...
2,Political temperatures rise and vicious storms...,huff,“The Gulf” is a story told in storms ― gales a...,'The Gulf' Is A Climate Change Novel For A Cha...
3,"""Beto O’Rourke will need to answer why he did ...",huff,Presidential candidate Beto O’Rourke defended ...,Beto O’Rourke Defends $5 Trillion Climate Plan...
4,The party is pushing for a vote this week that...,huff,The United Kingdom’s Labour Party is ramping u...,UK Labour Party Ratchets Up Climate Fight As G...
5,The Sunrise Movement announced its support for...,huff,The Green New Dealers are making their first e...,Green New Deal Activists Make First 2020 Endor...
6,The Vermont senator stood out in 2016 for his ...,huff,Sen. Bernie Sanders (I-Vt.) envisions a Green ...,Bernie Sanders Promises Green New Deal Jobs Wi...
7,The Massachusetts senator took a shot at EPA c...,huff,Getty Editorial Democratic presidential hopefu...,Elizabeth Warren On Climate Plan: 'No Coal Lob...
8,The Washington state governor and presidential...,huff,Jay Inslee is goading his rivals for the 2020 ...,Jay Inslee Is Calling On 2020 Rivals To Demand...
9,Nearly 2 in 3 Americans cool their homes with ...,huff,Richard Drew/The Associated Press A shopper lo...,This Is Why These States Aren’t So Chill About...


Now that I have dataframes for both Fox News and Huffington Post, I can combine them. However, before doing so I will remove some rows from the Huffington Post (285) to match the number of rows in the Fox News dataframe (222). I do this because I want a balanced dataset. The trimmed dataframe is called new_huff.

In [2016]:
new_huff=huff[0:222]
new_huff

,description,publisher,text,title
0,The former congressman had defended taking con...,huff,Following criticism of his climate change plan...,Beto O’Rourke Will Sign 'No Fossil Fuel Money'...
1,"""Any city along the coast or on the ocean will...",huff,The plan drew some immediate criticism from ot...,Beto O'Rourke Urges Climate Change Action Duri...
2,Political temperatures rise and vicious storms...,huff,“The Gulf” is a story told in storms ― gales a...,'The Gulf' Is A Climate Change Novel For A Cha...
3,"""Beto O’Rourke will need to answer why he did ...",huff,Presidential candidate Beto O’Rourke defended ...,Beto O’Rourke Defends $5 Trillion Climate Plan...
4,The party is pushing for a vote this week that...,huff,The United Kingdom’s Labour Party is ramping u...,UK Labour Party Ratchets Up Climate Fight As G...
5,The Sunrise Movement announced its support for...,huff,The Green New Dealers are making their first e...,Green New Deal Activists Make First 2020 Endor...
6,The Vermont senator stood out in 2016 for his ...,huff,Sen. Bernie Sanders (I-Vt.) envisions a Green ...,Bernie Sanders Promises Green New Deal Jobs Wi...
7,The Massachusetts senator took a shot at EPA c...,huff,Getty Editorial Democratic presidential hopefu...,Elizabeth Warren On Climate Plan: 'No Coal Lob...
8,The Washington state governor and presidential...,huff,Jay Inslee is goading his rivals for the 2020 ...,Jay Inslee Is Calling On 2020 Rivals To Demand...
9,Nearly 2 in 3 Americans cool their homes with ...,huff,Richard Drew/The Associated Press A shopper lo...,This Is Why These States Aren’t So Chill About...


Now that I have both dataframes for Fox News and the Huffington Post each containing the same features, I can combine them into one dataframe. The combined dataframe can be seen below and will be used for my analysis. In total, there are 444 rows since the Fox News and Huffington Post dataframs have 222 rows. 

In [2694]:
#append both df
combine=fox.append(new_huff, ignore_index=True)
combine

,description,publisher,text,title
0,Democratic presidential candidate Beto O'Rourk...,fox,Remember Beto?\n\nMonday former congressman fr...,Gutfeld on Beto's $5 trillion plan to save the...
1,"""The Five"" host Jesse Watters had some tough w...",fox,"""The Five"" co-host Jesse Watters had some toug...",Watters on Beto's climate plan: He's a 'scam a...
2,Democrats who truly wish to address climate an...,fox,"This week, Democrats in the House of Represent...",Mandy Gunasekara: Democrats should back Trump'...
3,In the next few weeks California Gov. Gavin Ne...,fox,"In the next few weeks, California Gov. Gavin N...",California Gov. Newsom facing new pressure fro...
4,Climate change activists pulled an Earth Day s...,fox,Here’s a headline for you: California climate ...,Tomi Lahren: California is the Golden State of...
5,Actor Alec Baldwin said Tuesday that his passi...,fox,Actor Alec Baldwin said Tuesday that his passi...,Alec Baldwin calls for action on climate chang...
6,"Little has changed over at CBS, with the netwo...",fox,CBS dug up for Earth Day a decades-old news cl...,CBS resurfaces 1970 'Act or Die' alarmist clim...
7,The hurricane pre-season hysteria is ramping u...,fox,The hurricane pre-season hysteria is ramping u...,"Joe Bastardi: Hurricanes happen, whether they ..."
8,Climate-change activists took over the Univers...,fox,Climate-change activists took over the Univers...,Protesters climb Universal Studios globe in Ca...
9,The protesters who glued themselves to trains ...,fox,The protesters who glued themselves to trains ...,Climate change protesters announce shift to po...


Below, I save the dataframe to a csv file for backup. 

In [2695]:
#save to csv
combine.to_csv("/Users/halabanz/Desktop/fox_huff-scrape.csv", sep=',')